In [1]:
import os
import gc
from monai.transforms import LoadImaged, EnsureChannelFirstd, ResampleToMatchd, Orientationd, Compose
import monai
import numpy as np
from tqdm import tqdm
import multiprocessing
from ipywidgets import interactive, widgets, fixed
from matplotlib import animation, rc; rc('animation', html='jshtml')
import matplotlib.pyplot as plt
%matplotlib inline

c:\Users\Justine Huang\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  device: torch.device = torch.device("cpu"),


In [17]:
image_dir = "D:/Shiny_icarus/Original/"
mask_dir = "D:/Shiny_icarus/Label/"
mask_list = os.listdir(mask_dir)
image_list = os.listdir(image_dir)

In [18]:
transform = Compose(
    [
        LoadImaged(reader=("nibabelreader", "nibabelreader"), keys=["image", "seg"]),
        EnsureChannelFirstd(keys=["image", "seg"]),
        # unify the orientations of image and mask
        Orientationd(keys=["image", "seg"], axcodes="RAS"),
    ]
)

In [19]:
def create_animation(img, seg, seg_rev=False, fps=10):

    images = img
    segs = seg
    
    if seg_rev:
        segs = segs[::-1]
    ims_sgs = [np.concatenate([images[i], segs[i]], axis=1) for i in range(len(images))]
    
    # Stack images
    animation_arr = np.stack(ims_sgs, axis=0)
    
    del images, ims_sgs
    gc.collect()
    
    # Initialise plot
    fig = plt.figure(figsize=(5,5), dpi=160)  # if size is too big then gif gets truncated
    im = plt.imshow(animation_arr[0], cmap='bone')
    plt.axis('off')
    
    # Load next frame
    def animate_func(i):
        im.set_array(animation_arr[i])
        return [im]
    plt.close()
    
    # Animation function
    anim = animation.FuncAnimation(fig, animate_func, frames = animation_arr.shape[0], interval = 1000//fps)
    
    return anim

In [21]:
name1 = "147_0000"
name2 = "147_0001"
img_sample = f"{name1}.nii.gz"
mask_sample = f"{name2}.nii.gz"
img_sample
# data = {"image": os.path.join(image_dir, img_sample), "seg": os.path.join(mask_dir, mask_sample)}
# output = transform(data)

# img = output["image"].numpy().transpose([0, 3, 2, 1])[0]
# seg = output["seg"].numpy().transpose([0, 3, 2, 1])[0]

# img = (img-np.min(img))/(np.max(img)-np.min(img)+1e-6)
# img = (img*255).astype(np.uint8)

# seg = np.where(seg>0, 255, 0).astype(np.uint8)


# create_animation(img, seg, fps=30)

'147_0000.nii.gz'